In [1]:
!pip install nltk

In [2]:
pip install numpy --only-binary=:all:

Note: you may need to restart the kernel to use updated packages.


In [8]:
# Import standard libraries
import os, io
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# Azure Cosmos DB client
from azure.cosmos import CosmosClient

# Azure Blob Storage client
import nltk

from azure.storage.blob import BlobServiceClient


# Deep learning libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout, Input, concatenate
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image

import torch
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertForSequenceClassification, Trainer, TrainingArguments

# Ensure NLTK data is downloaded
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\khush\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\khush\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [12]:
from dotenv import dotenv_values


config    = dotenv_values(".env")
COSMOS_URL = config["COSMOS_ENDPOINT"]
COSMOS_KEY = config["COSMOS_KEY"]
DATABASE_NAME = config["COSMOS_DATABASE"]
CONTAINER_NAME = config["COSMOS_CONTAINER"]

# Connect to Cosmos DB
cosmos_client = CosmosClient(COSMOS_URL, credential=COSMOS_KEY)
database = cosmos_client.get_database_client(DATABASE_NAME)
container = database.get_container_client(CONTAINER_NAME)

# Query all items (select relevant fields)
query = "SELECT c.id, c.statement, c.image, c.web, c.category, c.date, c.label FROM c"
items = list(container.query_items(
    query=query,
    enable_cross_partition_query=True
))
df = pd.DataFrame(items)
print(f"Loaded {len(df)} records from Cosmos DB.")
df.head()

Loaded 26599 records from Cosmos DB.


,id
0,0
1,1
2,3
3,4
4,5


In [13]:
import os
from azure.storage.blob import BlobServiceClient

# Use your Azure Storage connection string
CONNECTION_STRING =config["STORAGE_CONN_STR"]
BLOB_CONTAINER = config["STORAGE_CONTAINER"]

# Connect to Blob service via connection string
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)
container_client = blob_service_client.get_container_client(BLOB_CONTAINER)

# Download all blobs (or only those referenced by df) into a local folder
image_dir = "downloaded_images"
os.makedirs(image_dir, exist_ok=True)

for blob in container_client.list_blobs():
    blob_client = container_client.get_blob_client(blob)
    download_path = os.path.join(image_dir, blob.name)
    with open(download_path, "wb") as f:
        f.write(blob_client.download_blob().readall())

print(f"Downloaded {len(list(container_client.list_blobs()))} blobs to '{image_dir}'.")


Incomplete download: ('Connection broken: IncompleteRead(15721 bytes read, 105837 more expected)', IncompleteRead(15721 bytes read, 105837 more expected))
Incomplete download: ('Connection broken: IncompleteRead(32105 bytes read, 196197 more expected)', IncompleteRead(32105 bytes read, 196197 more expected))


ServiceRequestError: <urllib3.connection.HTTPSConnection object at 0x0000029B9F614340>: Failed to resolve 'infocrucible.blob.core.windows.net' ([Errno 11001] getaddrinfo failed)

In [14]:
import re
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words('english'))

def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)   # remove punctuation
    tokens = nltk.word_tokenize(text)
    tokens = [stemmer.stem(t) for t in tokens if t not in stop_words and t.isalpha()]
    return " ".join(tokens)

# Apply normalization
df['clean_statement'] = df['statement'].astype(str).apply(normalize_text)
df = df.dropna(subset=['clean_statement'])
print("Sample normalized text:", df['clean_statement'].iloc[0])



NameError: name 'stopwords' is not defined

In [15]:
# Remove exact duplicate statements
before = len(df)
df = df.drop_duplicates(subset=['clean_statement']).reset_index(drop=True)
after = len(df)
print(f"Dropped {before-after} duplicate statements; {after} remain.")


KeyError: Index(['clean_statement'], dtype='object')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Separate real vs fake for augmentation
real_df = df[df['label']=='real'].reset_index(drop=True)
fake_df = df[df['label']=='fake'].reset_index(drop=True)

# Vectorize fake statements with TF-IDF
tfidf_vec = TfidfVectorizer()
fake_tfidf = tfidf_vec.fit_transform(fake_df['clean_statement'])

# Compute cosine similarity among fake statements
sim_matrix = cosine_similarity(fake_tfidf)

# Threshold for similarity (e.g., 0.8); combine sentences with high similarity
augmented_texts = []
threshold = 0.8
for i in range(len(fake_df)):
    for j in range(i+1, len(fake_df)):
        if sim_matrix[i,j] > threshold:
            # Combine unique words from both sentences
            words_i = set(fake_df.loc[i, 'clean_statement'].split())
            words_j = set(fake_df.loc[j, 'clean_statement'].split())
            combined = " ".join(list(words_i.union(words_j)))
            augmented_texts.append(combined)

# Create new fake entries from augmented texts
aug_df = pd.DataFrame({
    'id': [f"aug_{k}" for k in range(len(augmented_texts))],
    'clean_statement': augmented_texts,
    'image': None,
    'web': None,
    'category': None,
    'date': None,
    'label': 'fake'
})
print(f"Generated {len(augmented_texts)} augmented fake statements.")
df_augmented = pd.concat([df, aug_df], ignore_index=True)


In [ ]:
# Tokenize for LDA
tokenized = [text.split() for text in df_augmented['clean_statement']]
dictionary = corpora.Dictionary(tokenized)
corpus = [dictionary.doc2bow(text) for text in tokenized]

# Train LDA model (e.g., 5 topics)
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)
for idx, topic in lda_model.print_topics(num_words=5):
    print(f"Topic {idx}: {topic}")


In [ ]:
# TF-IDF features (unigram)
tfidf = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf.fit_transform(df_augmented['clean_statement']).toarray()

# Sentence embeddings using DistilBERT
st_model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
X_emb = st_model.encode(df_augmented['clean_statement'], show_progress_bar=True)
print(f"Text embeddings shape: {X_emb.shape}")


In [ ]:
# Load ResNet-50 model for feature extraction (no classification head)
resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_image_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = resnet.predict(x)
    return features.flatten()

# Map image filenames to features
img_features = {}
for fname in df_augmented['image'].dropna().unique():
    path = os.path.join(image_dir, fname)
    if os.path.exists(path):
        img_features[fname] = extract_image_features(path)
print(f"Extracted features for {len(img_features)} images.")


In [ ]:
# Merge image features into dataframe (for statements with images)
df_augmented['img_feat'] = df_augmented['image'].map(lambda fname: img_features.get(fname))
# Fill missing image features with zeros (for simplicity)
no_image_vec = np.zeros(resnet.output_shape[-1])
df_augmented['img_feat'] = df_augmented['img_feat'].apply(lambda x: x if isinstance(x, np.ndarray) else no_image_vec)

# Feature matrices
X_text_tfidf = X_tfidf  # (n_samples, n_features)
X_text_emb = X_emb     # (n_samples, emb_dim)
X_image = np.stack(df_augmented['img_feat'].values)
y = (df_augmented['label'] == 'fake').astype(int).values  # binary labels

# Combine TF-IDF and image for one set, embeddings and image for another, etc.
X_tfidf_img = np.hstack([X_text_tfidf, X_image])
X_emb_img = np.hstack([X_text_emb, X_image])

# Train/test split
X_tfidf_img_train, X_tfidf_img_test, y_train, y_test = train_test_split(X_tfidf_img, y, test_size=0.2, random_state=42)
X_emb_img_train, X_emb_img_test, _, _ = train_test_split(X_emb_img, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

# Logistic Regression on TF-IDF alone
lr = LogisticRegression(max_iter=1000)
lr.fit(X_text_tfidf[y_train.index], y_train)   # text only (if we separated earlier)
y_pred_lr = lr.predict(X_text_tfidf[y_test.index])
print("Logistic Regression (text only):", accuracy_score(y_test, y_pred_lr))

# Decision Tree on TF-IDF + image
dt = DecisionTreeClassifier()
dt.fit(X_tfidf_img_train, y_train)
y_pred_dt = dt.predict(X_tfidf_img_test)
print("Decision Tree (TF-IDF+image):", accuracy_score(y_test, y_pred_dt))

# Naive Bayes on TF-IDF
nb = MultinomialNB()
nb.fit(X_tfidf_img_train, y_train)  # Note: MultinomialNB expects non-negative features
y_pred_nb = nb.predict(X_tfidf_img_test)
print("Naive Bayes (TF-IDF+image):", accuracy_score(y_test, y_pred_nb))


In [ ]:
# Prepare text sequences for LSTM
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df_augmented['clean_statement'])
seqs = tokenizer.texts_to_sequences(df_augmented['clean_statement'])
seqs_padded = pad_sequences(seqs, maxlen=100)
X_seq = seqs_padded
X_seq_train, X_seq_test, _, _ = train_test_split(X_seq, y, test_size=0.2, random_state=42)

# LSTM model
input_layer = Input(shape=(100,))
x = Embedding(input_dim=10000, output_dim=128)(input_layer)
x = LSTM(64)(x)
x = Dropout(0.5)(x)
output_layer = Dense(1, activation='sigmoid')(x)
lstm_model = Model(input_layer, output_layer)
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(X_seq_train, y_train, epochs=2, batch_size=32, validation_split=0.1)  # brief training
y_pred_lstm = (lstm_model.predict(X_seq_test) > 0.5).astype(int)
print("LSTM Accuracy:", accuracy_score(y_test, y_pred_lstm))


In [ ]:
# Bi-LSTM model
input_layer = Input(shape=(100,))
x = Embedding(input_dim=10000, output_dim=128)(input_layer)
x = Bidirectional(LSTM(64))(x)
x = Dropout(0.5)(x)
output_layer = Dense(1, activation='sigmoid')(x)
bilstm_model = Model(input_layer, output_layer)
bilstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
bilstm_model.fit(X_seq_train, y_train, epochs=2, batch_size=32, validation_split=0.1)
y_pred_bilstm = (bilstm_model.predict(X_seq_test) > 0.5).astype(int)
print("Bi-LSTM Accuracy:", accuracy_score(y_test, y_pred_bilstm))


In [ ]:
# Example: Use pre-trained VGG16 for image classification (pseudo-code)
from tensorflow.keras.applications.vgg16 import VGG16
vgg = VGG16(weights='imagenet', include_top=False, pooling='avg')
# Extract features for images and train a small neural network classifier on top


In [ ]:
from tensorflow.keras.layers import Concatenate

# Example MLP on combined text embedding + image feature
text_input = Input(shape=(X_emb.shape[1],))
img_input = Input(shape=(X_image.shape[1],))
merged = Concatenate()([text_input, img_input])
x = Dense(256, activation='relu')(merged)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)
mlp = Model([text_input, img_input], output)
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Prepare inputs for fusion model
X_comb_train = [X_text_emb[y_train.index], X_image[y_train.index]]
X_comb_test = [X_text_emb[y_test.index], X_image[y_test.index]]
mlp.fit(X_comb_train, y_train, epochs=2, batch_size=32, validation_split=0.1)
y_pred_mlp = (mlp.predict(X_comb_test) > 0.5).astype(int)
print("Fusion MLP Accuracy:", accuracy_score(y_test, y_pred_mlp))


In [ ]:
from sklearn.metrics import classification_report

print("Classification Report for Fusion MLP:")
print(classification_report(y_test, y_pred_mlp))
cm = confusion_matrix(y_test, y_pred_mlp)
print("Confusion Matrix:\n", cm)


In [ ]:
# Insert documents into Azure Cosmos Vector DB
from azure.cosmos import CosmosClient
from azure.cosmos.partition_key import PartitionKey

# Create a container with vector indexing enabled (requires Azure setup)
vector_container = database.get_container_client("vector_store")
for idx, row in df_augmented.iterrows():
    embedding = st_model.encode(row['clean_statement']).tolist()
    vector_container.upsert_item({
        'id': str(row['id']),
        'statement': row['clean_statement'],
        'label': row['label'],
        'embedding': embedding  # stored as a vector type
    })

#  RAG Query: Given a statement, retrieve similar docs and call LLM
def rag_fact_check(query_text):
    query_emb = st_model.encode([query_text]).tolist()[0]
    sql_query = {
        'query': "SELECT TOP 3 c.statement FROM c ORDER BY ST_DISTANCE(c.embedding, @q) ASC",
        'parameters': [{'name': '@q', 'value': query_emb}]
    }
    docs = list(vector_container.query_items(query=sql_query, enable_cross_partition_query=True))
    context = "\n\n".join([d['statement'] for d in docs])
    # Use OpenAI Completion with context
    from openai import OpenAI
    client = OpenAI(api_key="<YOUR_OPENAI_KEY>")
    response = client.chat.completions.create(
        model="gpt-4o-preview",
        messages=[
            {"role":"system", "content": "Check the statement using the context."},
            {"role":"user", "content": f"Statement: {query_text}\nContext:{context}"}
        ]
    )
    return response.choices[0].message.content


